# Learning to use the Spotify API

I am using API token instead of their in-house library Spotipy, as I was having some issues with the OAuth of the same.


This notebook can help you to search queries for tracks, artist and albums.
YOu can get a client ID and client secret by going to SPotify's developer dashboard and creating a new WebApp.

In [15]:
import requests
import datetime
from urllib.parse import urlencode

In [17]:
import base64
client_id = '' #put your client_id here
client_secret = '' #put your client_secret here

In [18]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    


In [19]:
spotify = SpotifyAPI(client_id, client_secret)


In [22]:
query_response = spotify.search({"track": "Time"}, search_type="track")

q=track%3ATime&type=track


In [23]:
import pandas as pd    

df = pd.json_normalize(query_response)

In [31]:
df

,tracks.href,tracks.items,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total
0,https://api.spotify.com/v1/search?query=track%...,"[{'album': {'album_type': 'single', 'artists':...",20,https://api.spotify.com/v1/search?query=track%...,0,None,10000


In [29]:
track_item_df =  pd.json_normalize(df['tracks.items'][0])

In [30]:
track_item_df

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,206864,False,https://api.spotify.com/v1/tracks/6KGtAlYvgm4W...,6KGtAlYvgm4W9PoYabFAYE,False,One Thing At A Time,81,...,04VpyPCr9PQRWRRp1ecOs1,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",One Thing At A Time (Sampler),2022-12-02,day,3,album,spotify:album:04VpyPCr9PQRWRRp1ecOs1,USUG12209213,https://open.spotify.com/track/6KGtAlYvgm4W9Po...
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,182306,False,https://api.spotify.com/v1/tracks/2R4AlwtrrkMa...,2R4AlwtrrkMaRKojcTIzmL,False,For the First Time,83,...,4NNq2vwTapv4fSJcrZbPH7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",This Old Dog,2017-05-05,day,13,album,spotify:album:4NNq2vwTapv4fSJcrZbPH7,QMMZN1701301,https://open.spotify.com/track/2R4AlwtrrkMaRKo...
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,151933,False,https://api.spotify.com/v1/tracks/5hslUAKq9I9C...,5hslUAKq9I9CG2bAulFkHN,False,It's the Most Wonderful Time of the Year,87,...,4dcJHxlOJwtyypL7sx1qch,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Andy Williams Christmas Album,1963-11-24,day,12,album,spotify:album:4dcJHxlOJwtyypL7sx1qch,USSM16300086,https://open.spotify.com/track/5hslUAKq9I9CG2b...
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,191822,True,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,1PckUlxKqWQs3RlWXVBLw3,False,About Damn Time,86,...,4JApkbiesQxv5drdEZRlZC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",About Damn Time,2022-04-14,day,1,album,spotify:album:4JApkbiesQxv5drdEZRlZC,USAT22202139,https://open.spotify.com/track/1PckUlxKqWQs3Rl...
4,[{'external_urls': {'spotify': 'https://open.s...,"[AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, BR, C...",1,214190,False,https://api.spotify.com/v1/tracks/44qEAOL7eiNJ...,44qEAOL7eiNJplZI5B9rND,False,Out of Time,3,...,27Rz5rm7oJke4AQ8LLPexT,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Synchronized Heartbeats,2023-01-20,day,19,album,spotify:album:27Rz5rm7oJke4AQ8LLPexT,USUG12106689,https://open.spotify.com/track/44qEAOL7eiNJplZ...
5,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,168920,True,https://api.spotify.com/v1/tracks/0YFqKxV9uNu6...,0YFqKxV9uNu6LUeYkLOKRS,False,On Time (with John Legend),81,...,7txGsnDSqVMoRl6RQ9XyZP,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",HEROES & VILLAINS,2022-12-02,day,15,album,spotify:album:7txGsnDSqVMoRl6RQ9XyZP,USUG12208782,https://open.spotify.com/track/0YFqKxV9uNu6LUe...
6,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,214193,False,https://api.spotify.com/v1/tracks/2SLwbpExuoBD...,2SLwbpExuoBDZBpjfefCtV,False,Out of Time,83,...,2nLOHgzXzwFEpl62zAgCEC,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Dawn FM,2022-01-06,day,16,album,spotify:album:2nLOHgzXzwFEpl62zAgCEC,USUG12106689,https://open.spotify.com/track/2SLwbpExuoBDZBp...
7,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AM, AO, AR, AT, AU, AZ, BB, BD, B...",1,156081,True,https://api.spotify.com/v1/tracks/240audWazVjw...,240audWazVjwvwh7XwfSZE,False,For the Last Time,76,...,4CcyRnFW6Vgf372ca3xFGR,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",For the Last Time,2017-09-05,day,1,album,spotify:album:4CcyRnFW6Vgf372ca3xFGR,QM8DG1703420,https://open.spotify.com/track/240audWazVjwvwh...
8,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BA, BD, BE, BG, BN, BO, B...",1,163093,False,https://api.spotify.com/v1/tracks/3bMfUUpzUzqW...,3bMfUUpzUzqWp9II00yRhE